# Project 4 - Part 2: Cleaning and feature extraction

## [2.1 Importing and loading data](#2.1)
### [2.1.1 Import packages](#2.1.1)
### [2.1.2 Loading and inspecting data](#2.1.2)

## [2.2 Extracting and cleaning data](#2.2)
### [2.2.1 Target variable: salary](#2.2.1)
### [2.2.2 Job title keywords](#2.2.2)
### [2.2.3 Location](#2.2.3)
### [2.2.4 Summary keywords](#2.2.4)
### [2.2.5 Manually creating skill variables](#2.2.5)

## [2.3 Final cleaning and exporting](#2.3)

## <a id='2.1'>2.1 Importing and loading data</a>

### <a id='2.1.1'>2.1.1 Import packages</a>

In [5]:
import pandas as pd
import numpy as np
import re
import string

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### <a id='2.1.2'>2.1.2 Loading and inspecting data</a>

In [6]:
#Read the job data csv created in the webscraping notebook
file = 'jobs.csv'
jobs = pd.read_csv(file)
jobs.head(2)

,title,company,location,salary,summary
0,Data Engineer,Telstra,Sydney NSW,"Full-time, Permanent",Telstra’s vision is to become a world class te...
1,Junior Data Analyst/Scientist,International Institute of Data & Analytics,Sydney NSW,No info,The International Institute of Data & Analytic...


In [7]:
#What are the data types? Expect all objects
jobs.dtypes

title       object
company     object
location    object
salary      object
summary     object
dtype: object

## <a id='2.2'>2.2 Extracting and cleaning data</a>

### <a id='2.2.1'>2.2.1 Target variable: salary</a>

In [8]:
#Change those 'No info' values in salary to NaN
jobs['salary'] = jobs['salary'].apply(lambda x: np.nan if x == 'No info' else x)

In [9]:
#What columns can we pull out from the salary column?
jobs['salary'].unique()[:20]

array(['Full-time,\xa0Permanent', nan, 'Contract',
       '\r\n                $80,000 - $100,000 a year',
       '\r\n                $73,300 - $108,400 a year',
       '\r\n                $91,743 - $98,127 a year - ',
       '\r\n                $70,000 - $80,000 a year',
       '\r\n                $100,000 - $150,000 a year',
       '\r\n                $57,534 - $76,712 a year',
       '\r\n                $120,000 - $160,000 a year - ',
       '\r\n                $70,000 - $90,000 a year',
       '\r\n                $100,000 - $120,000 a year - ', 'Internship',
       '\r\n                $100,000 a year',
       '\r\n                $109,000 - $128,000 a year', 'Permanent',
       '\r\n                $110,000 a year',
       '\r\n                $100,000 - $150,000 a year - ',
       'Part-time,\xa0Temporary,\xa0Contract,\xa0Internship',
       '\r\n                $70 - $80 an hour - '], dtype=object)

In [10]:
#Create regular expressions to extract the low and high numbers from jobs that list an annual salary range, return same for both
#columns if only one value is listed
low_re = '\$\d\S+\s\-'
high_re = '\-\s\$\d\S+\s'
other_re = '[^-]\s\$\d\S+\s(?!-)'

jobs['salary_low'] = [re.findall(low_re, str(i)) if len(re.findall(low_re, str(i))) > 0 else re.findall(other_re, str(i)) for i in jobs['salary']]
jobs['salary_high'] = [re.findall(high_re, str(i)) if len(re.findall(high_re, str(i))) > 0 else re.findall(other_re, str(i)) for i in jobs['salary']]

In [11]:
#Now get the values out of lists, remove unwanted characters and convert columns to floats
unwanted = '$,- '
removechars = str.maketrans('','',unwanted)
for i in range(len(jobs['salary_low'])):
    try:
        jobs['salary_low'][i] = jobs['salary_low'][i][0].translate(removechars)
    except:
        jobs['salary_low'][i] = np.nan
for i in range(len(jobs['salary_high'])):
    try:
        jobs['salary_high'][i] = jobs['salary_high'][i][0].translate(removechars)
    except:
        jobs['salary_high'][i] = np.nan
        
jobs['salary_low'] = jobs['salary_low'].astype('float')
jobs['salary_high'] = jobs['salary_high'].astype('float')


In [12]:
#There are quite a few duplicate rows, possibly ones that came up in both searches I conducted in the scraping. 
#I'll drop duplicate rows
jobs = jobs.drop_duplicates()
jobs = jobs.reset_index()
jobs.shape

(745, 8)

In [13]:
jobs.describe()

,index,salary_low,salary_high
count,745.000000,135.000000,135.000000
mean,604.127517,76766.273333,89353.584444
std,376.799697,44381.960316,56655.377737
min,0.000000,35.000000,36.900000
25%,271.000000,60500.000000,70434.000000
50%,548.000000,80000.000000,90000.000000
75%,946.000000,102492.500000,120000.000000
max,1255.000000,200000.000000,300000.000000


In [14]:
#Some of the salary information is listed by month
jobs[jobs['salary'].str.contains('month')==True]

,index,title,company,location,salary,summary,salary_low,salary_high
277,410,CSIRO Undergraduate Vacation Scholarships - CS...,CSIRO,Melbourne VIC,"\r\n $3,074 a month",Work at the interface between science and indu...,3074.0,3074.0
314,460,CSIRO Undergraduate Vacation Scholarships - Bu...,CSIRO,Australia,"\r\n $3,074 a month",Get hands-on work experience and gain new skil...,3074.0,3074.0


In [15]:
#We'll convert these to annual rates by multiplying by 12
mask = jobs[jobs['salary'].str.contains('month')==True].index
jobs['salary_low'][mask] = jobs['salary_low'][mask].apply(lambda x: x * 12)
jobs['salary_high'][mask] = jobs['salary_high'][mask].apply(lambda x: x * 12)

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [16]:
#Some are weekly
jobs[jobs['salary'].str.contains('week')==True]

,index,title,company,location,salary,summary,salary_low,salary_high
255,375,Indigenous Cadetships - Data61 Mathematics & S...,CSIRO,Australia,\r\n $590 a week,Improve your professional employment prospects...,590.0,590.0
296,435,Indigenous Cadetships - Data61 Computer Science,CSIRO,Australia,\r\n $590 a week,Improve your professional employment prospects...,590.0,590.0
361,528,Indigenous Cadetship - Data61 Cyber Security Risk,CSIRO,Melbourne VIC,\r\n $590 a week,Improve your professional employment prospects...,590.0,590.0
463,784,Data Analyst,Robert Half Australia,Sydney NSW,\r\n $500 a week -,6-month Data Analyst role working with Global ...,500.0,500.0
539,921,Reg Reporting - Data Analyst,Morgan McKinley,Sydney Central Business District NSW,\r\n $600 - $800 a week -,Reg reporting in the FS space\r\n\r\nDescripti...,600.0,800.0
544,926,Test Analyst - Data Migration,Hays,Sydney Central Business District NSW,\r\n $600 - $700 a week -,"Test Analyst - Data Migration, Sydney CBD, $60...",600.0,700.0
677,1133,Data Scientist,Robert Half Australia,Sydney NSW,\r\n $800 a week,Define how the distributed structure will work...,800.0,800.0


In [17]:
#We'll just convert these to something annual by multiplying by 52
mask = jobs[jobs['salary'].str.contains('week')==True].index
jobs['salary_low'][mask] = jobs['salary_low'][mask].apply(lambda x: x * 52)
jobs['salary_high'][mask] = jobs['salary_high'][mask].apply(lambda x: x * 52)

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [18]:
#Some by day
jobs[jobs['salary'].str.contains('day')==True]

,index,title,company,location,salary,summary,salary_low,salary_high
116,164,Data Scientist - Ecommerce,Greythorn Specialist Technology Recruitment,Sydney NSW,\r\n $900 a day -,Marketing leading Financial Service company se...,900.0,900.0
129,187,Analytics Lead / Data Scientist,Charterhouse,New South Wales,\r\n $530 a day,Job details\r\nLocation: NSW\r\n\r\nDiscipline...,530.0,530.0
131,189,Data Scientist - Ecommerce,Greythorn Australia,Sydney NSW,\r\n $900 a day -,$900 per day plus super. Join an EEO employer\...,900.0,900.0
132,190,Data Scientist - Ecommerce,Greythorn Australia Old,Sydney NSW,\r\n $900 a day -,$900 per day plus super. Join an EEO employer\...,900.0,900.0
197,287,"Agile Business Analyst - AI, Machine Learning,...",Clear Recruitment,Sydney NSW,\r\n $750 a day -,The Client:\r\n\r\nOur client is an innovative...,750.0,750.0
458,777,Data Analyst (SAS/ SQL),Peoplebank,Sydney Central Business District NSW,\r\n $500 - $700 a day -,Role:\r\nOur client is a leading financial ser...,500.0,700.0
466,793,Data Analyst,Talenza,Sydney NSW,\r\n $620 a day -,Work with the best in the business\r\n\r\n$620...,620.0,620.0
491,835,Data Analyst,Talent International,New South Wales,\r\n $400 - $450 a day -,Analyst - 3-6 Month Contract - Data Insight\r\...,400.0,450.0
529,903,Business & Data Analyst,Hays,Sydney Central Business District NSW,\r\n $650 a day -,Job for a Business Analyst at a government org...,650.0,650.0
540,922,ICT Business Analyst - Data Analyst,AUREC,Sydney NSW,\r\n $800 a day -,This is a great opportunity for an enthusiasti...,800.0,800.0


In [19]:
#We'll convert these to an annual rate by multiplying by the number of working days in a year. I'll assume that this is
#effectively 5 * 52.
workingdays = 5 * 52
mask = jobs[jobs['salary'].str.contains('day')==True].index
jobs['salary_low'][mask] = jobs['salary_low'][mask].apply(lambda x: x * workingdays)
jobs['salary_high'][mask] = jobs['salary_high'][mask].apply(lambda x: x * workingdays)

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
#Some per hour
jobs[jobs['salary'].str.contains('hour')==True]

,index,title,company,location,salary,summary,salary_low,salary_high
71,93,Data Scientist,Chandler Macleod,Perth WA,\r\n $70 - $80 an hour -,Short Term contract\r\nPerth CBD Location\r\nW...,70.0,80.0
110,154,Data Scientist,Charterhouse,New South Wales,\r\n $40 an hour,Job details\r\nLocation: NSW\r\n\r\nDiscipline...,40.0,40.0
387,638,Business Analyst/ Reporting Analyst,Ignite,Sydney NSW,\r\n $60 - $62 an hour -,Currently seeking a Business Analyst/ Reportin...,60.0,62.0
417,689,Data Analyst,Latrobe Community Health Service,Ringwood VIC,\r\n $36.90 an hour -,"Data Analyst\r\n\r\nFull Time, Fixed Term (unt...",36.9,36.9
709,1188,Research Analyst/Senior Analyst,Patternmakers,Melbourne VIC,\r\n $35 - $40 an hour -,- Opportunity for an analyst with exceptional ...,35.0,40.0


In [21]:
#These ones we'll convert to an annual rate by assuming 8 hours a day, for the same number of annual working days as above
mask = jobs[jobs['salary'].str.contains('hour')==True].index
jobs['salary_low'][mask] = jobs['salary_low'][mask].apply(lambda x: x * workingdays * 8)
jobs['salary_high'][mask] = jobs['salary_high'][mask].apply(lambda x: x * workingdays * 8)

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [22]:
#Since we have a range of salary values for many of the jobs, I'll summarize these two range values in an extra column
#The mean salary
jobs['salary_mean'] = (jobs['salary_low'] + jobs['salary_high']) / 2

In [23]:
#Now we can look at the stats for the salary again
jobs.describe()
#Looks better! Now I will start working on pulling out some feature columns.

,index,salary_low,salary_high,salary_mean
count,745.000000,135.000000,135.000000,135.000000
mean,604.127517,96706.703704,110150.696296,103428.700000
std,376.799697,40459.034901,49004.828766,43405.974525
min,0.000000,26000.000000,26000.000000,26000.000000
25%,271.000000,70434.000000,80000.000000,75876.000000
50%,548.000000,88713.000000,100000.000000,91807.500000
75%,946.000000,116756.500000,130000.000000,125000.000000
max,1255.000000,234000.000000,300000.000000,250000.000000


### <a id='2.2.2'>2.2.2 Job title keywords</a>

In [24]:
#Let's start by creating some categories from the job title column
#Our bag-of-words will be the 'title' column of our jobs dataframe
bow = jobs['title']

In [25]:
#Fit a Count Vectorizer
tvec = TfidfVectorizer(stop_words='english')
tvec.fit(bow)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [26]:
#Generate columns from the count vectorizer
tvecdf  = pd.DataFrame(tvec.transform(bow).todense(),
             columns=tvec.get_feature_names())

tvecdf.transpose().sort_values(0, ascending=False).transpose().head()

,engineer,data,10,permanent,perth,pharmacology,phd,phenotyping,php,physicist,...,ecologists,ecommerce,economics,economist,edi,education,el1,electricity,enablement,years
0,0.871845,0.489781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.339808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.581723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.581723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.581723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
#We probably don't want to include *all* the words as features, so I will reorder the columns by how many times each word
#appears in the corpus
tvecdf = tvecdf.reindex_axis(tvecdf.sum().sort_values(ascending=False).index, axis=1)
tvecdf.head().iloc[:,:30]

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,analyst,data,scientist,business,senior,research,engineer,lead,junior,developer,...,operations,intelligence,systems,software,sales,environmental,finance,consultant,digital,stack
0,0.000000,0.489781,0.000000,0.0,0.0,0.0,0.871845,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.268433,0.339808,0.475133,0.0,0.0,0.0,0.000000,0.0,0.765979,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.581723,0.813387,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.581723,0.813387,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.581723,0.813387,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
tvecdf.shape

(745, 624)

In [29]:
#Let's rename the columns so we know where they came from later
tvecdf.rename(columns = lambda x: 'title_'+str(x), inplace=True)

In [30]:
#Let's add the 20 most common terms to the jobs dataframe
top30 = tvecdf.iloc[:,:30]
jobs = jobs.join(top30)

In [31]:
jobs.head(2)

,index,title,company,location,salary,summary,salary_low,salary_high,salary_mean,title_analyst,...,title_operations,title_intelligence,title_systems,title_software,title_sales,title_environmental,title_finance,title_consultant,title_digital,title_stack
0,0,Data Engineer,Telstra,Sydney NSW,"Full-time, Permanent",Telstra’s vision is to become a world class te...,NaN,NaN,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Junior Data Analyst/Scientist,International Institute of Data & Analytics,Sydney NSW,NaN,The International Institute of Data & Analytic...,NaN,NaN,NaN,0.268433,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### <a id='2.2.3'>2.2.3 Location</a>

In [32]:
#Now to make some location-based columns
#First, note that we have a lot of variation in how particular locations are listed in the data. Some places are listed as 
#a city, such as Sydney. Others show the suburb. Almost all however, show a state. I will create state-level location
#columns from this column
jobs['location'].value_counts().head(20)

Sydney NSW                              365
Melbourne VIC                            72
Sydney Central Business District NSW     37
Australia                                31
Brisbane QLD                             27
Canberra ACT                             23
New South Wales                          21
Perth WA                                 14
Surry Hills NSW                           9
Melbourne City Centre VIC                 8
Queensland                                8
Macquarie Park NSW                        6
Darlinghurst NSW                          5
North Ryde NSW                            5
Sydney Western Suburbs NSW                5
Victoria                                  5
Parramatta NSW                            5
Parkville VIC                             4
Townsville QLD                            3
Newcastle NSW                             3
Name: location, dtype: int64

In [33]:
#Use a regex to pull out any strings of 2 or 3 capital letters (like in 'NSW' or 'SA'), or return the whole string if no such 
#string is present
state_re = '\s[A-Z]{2,3}'
jobs['state'] = [re.findall(state_re, str(i)) if len(re.findall(state_re, str(i))) > 0 else i for i in jobs['location']]

In [34]:
#Convert any lists to strings
for i in jobs.index:
    if len(jobs['state'][i]) < 4:
        jobs['state'][i] = jobs['state'][i][0]
    else:
        jobs['state'][i] = jobs['state'][i]

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
#Remove spaces
jobs['state'] = jobs['state'].apply(lambda x: x.replace(' ',''))

In [36]:
#Check unique values
jobs['state'].unique()

array(['NSW', 'QLD', 'WA', 'Queensland', 'NewSouthWales', 'VIC', 'ACT',
       'SA', 'Australia', 'Victoria', 'WesternAustralia', 'NT',
       'SouthAustralia', 'Tasmania'], dtype=object)

In [37]:
#Convert full length names to abbreviations
jobs['state'] = jobs['state'].map({'Queensland':'QLD','NewSouthWales':'NSW','Australia':'AUS','Victoria':'VIC',
                                  'WesternAustralia':'WA','SouthAustralia':'SA','Tasmania':'TAS',
                                  'NSW':'NSW','QLD':'QLD','WA':'WA','VIC':'VIC','ACT':'ACT','SA':'SA', 'NT':'NT'})

In [38]:
#Check unique values again
jobs['state'].unique()

array(['NSW', 'QLD', 'WA', 'VIC', 'ACT', 'SA', 'AUS', 'NT', 'TAS'],
      dtype=object)

In [39]:
#Make dummy columns for states
state_dummies = pd.get_dummies(jobs['state'])
state_dummies.rename(columns = lambda x: 'state_'+str(x), inplace=True)
state_dummies.sample(5)

,state_ACT,state_AUS,state_NSW,state_NT,state_QLD,state_SA,state_TAS,state_VIC,state_WA
149,0,0,1,0,0,0,0,0,0
244,0,0,1,0,0,0,0,0,0
325,0,0,1,0,0,0,0,0,0
688,0,1,0,0,0,0,0,0,0
578,0,0,1,0,0,0,0,0,0


In [40]:
#Add them to the dataframe
jobs = jobs.join(state_dummies)

### <a id='2.2.4'>2.2.4 Summary keywords</a>

In [41]:
#First clean up unwanted characters such as newline and - characters
unwanted = '\r\n-'
removechars = str.maketrans(' ',' ',unwanted)
jobs['summary'] = jobs['summary'].apply(lambda x: x.translate(removechars))

In [42]:
#I will look for important summary keywords using a Tf-idf vectorizer
from sklearn.feature_extraction import text
from nltk.stem import PorterStemmer, SnowballStemmer

I've elected to do stemming before extracting words from the summary since it is more likely that different variants of the same word will appear in the summary than the job title.

In [43]:
#First I'll do some stemming to take care of redundant word endings (e.g. 'experience' vs 'experienced')
stemmer = SnowballStemmer('english')
jobs['stemmed_summary'] = [[stemmer.stem(word) for word in sentence.split(" ")] for sentence in jobs['summary']]

In [44]:
for i in jobs['stemmed_summary'].index:
    jobs['stemmed_summary'][i] = str(jobs['stemmed_summary'][i])

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
stop_words = text.ENGLISH_STOP_WORDS.union(['work','busi','role','look','build','strong','understand','grow','ensur',
                                           'join','key','requir','year','help','www','weeks','deal','mid','exist','datayou'])
tvec = TfidfVectorizer(stop_words=stop_words)
tvec.fit(jobs['stemmed_summary'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=frozenset({'somewhere', 'where', 'we', 'least', 'against', 'bottom', 'mostly', 'bill', 'thence', 'everywhere', 'not', 'almost', 'now', 'system', 'how', 'twenty', 'wherever', 'within', 'the', 'himself', 'strong', 'though', 'for', 'must', 'further', 'requir', 'serious', 'found', 'throughout...row', 'around', 'be', 'wherein', 'towards', 'yourself', 'sincere', 'so', 'during', 'both', 'among'}),
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [46]:
tvecdf  = pd.DataFrame(tvec.transform(jobs['stemmed_summary']).todense(),
                   columns=tvec.get_feature_names())

tvecdf.transpose().sort_values(0, ascending=False).head(10).transpose()
#Order columns based on the total sum of tf-idf scores
tvecdf = tvecdf.reindex_axis(tvecdf.mean().sort_values(ascending=False).index, axis=1)

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  


In [47]:
tvecdf.shape

(745, 18022)

In [48]:
#Some of these seem like sensible words to look at (like, business, analytics, research, customer etc.). While others
#are likely to be uninformative (e.g work, role, ability) I will look through the top 30 and manually drop columns that seem
#uninformative.
tvecdf.head().iloc[:,:20]

,data,team,experi,manag,develop,analyt,support,research,client,custom,report,market,product,provid,project,skill,use,analyst,opportun,model
0,0.246799,0.051426,0.050113,0.020863,0.096553,0.045140,0.000000,0.000000,0.000000,0.028563,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.046115,0.025069,0.087606,0.000000
1,0.278075,0.065059,0.047548,0.000000,0.073289,0.171319,0.000000,0.092890,0.102245,0.000000,0.000000,0.0,0.025650,0.000000,0.048598,0.020444,0.000000,0.000000,0.000000,0.115354
2,0.064023,0.059292,0.034667,0.014432,0.013359,0.000000,0.000000,0.022575,0.000000,0.059277,0.017834,0.0,0.037402,0.000000,0.035432,0.000000,0.000000,0.000000,0.060603,0.063077
3,0.096435,0.026793,0.039163,0.016304,0.015091,0.105829,0.033927,0.051005,0.000000,0.000000,0.000000,0.0,0.021126,0.000000,0.000000,0.016838,0.036038,0.000000,0.017116,0.047505
4,0.132597,0.081865,0.015955,0.019927,0.073777,0.043115,0.041466,0.000000,0.000000,0.000000,0.049247,0.0,0.000000,0.042152,0.000000,0.020580,0.022023,0.000000,0.000000,0.058061


In [49]:
top1000 = tvecdf.iloc[:,:5000]
top1000.columns

Index(['data', 'team', 'experi', 'manag', 'develop', 'analyt', 'support',
       'research', 'client', 'custom',
       ...
       'solr', 'tesseract', 'problemsus', 'tobin', 'ediscoveri',
       'rehabilitation', 'entrepreneurs', 'courses', 'unconvent', 'conduit'],
      dtype='object', length=5000)

In [52]:
#Make a full table of keywords
summ_keywords = top1000#.join(extra)
#Ad the prefix kw_ to the column name to indicate that these are summary keywords
summ_keywords.rename(columns = lambda x: 'kw_'+str(x), inplace=True)
summ_keywords.head()

C:\Users\Chris\Anaconda3\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,kw_data,kw_team,kw_experi,kw_manag,kw_develop,kw_analyt,kw_support,kw_research,kw_client,kw_custom,...,kw_solr,kw_tesseract,kw_problemsus,kw_tobin,kw_ediscoveri,kw_rehabilitation,kw_entrepreneurs,kw_courses,kw_unconvent,kw_conduit
0,0.246799,0.051426,0.050113,0.020863,0.096553,0.045140,0.000000,0.000000,0.000000,0.028563,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.278075,0.065059,0.047548,0.000000,0.073289,0.171319,0.000000,0.092890,0.102245,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.064023,0.059292,0.034667,0.014432,0.013359,0.000000,0.000000,0.022575,0.000000,0.059277,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.096435,0.026793,0.039163,0.016304,0.015091,0.105829,0.033927,0.051005,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.132597,0.081865,0.015955,0.019927,0.073777,0.043115,0.041466,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
#Finally, add these keyword columns to the main dataframe
jobs = jobs.join(summ_keywords)

In [54]:
jobs.shape

(745, 5050)

In [55]:
jobs.head()

,index,title,company,location,salary,summary,salary_low,salary_high,salary_mean,title_analyst,...,kw_solr,kw_tesseract,kw_problemsus,kw_tobin,kw_ediscoveri,kw_rehabilitation,kw_entrepreneurs,kw_courses,kw_unconvent,kw_conduit
0,0,Data Engineer,Telstra,Sydney NSW,"Full-time, Permanent",Telstra’s vision is to become a world class te...,NaN,NaN,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Junior Data Analyst/Scientist,International Institute of Data & Analytics,Sydney NSW,NaN,The International Institute of Data & Analytic...,NaN,NaN,NaN,0.268433,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Data Scientist,Nearmap - AU,Barangaroo NSW,"Full-time, Permanent",Want to do petabyte scale deep learning and sh...,NaN,NaN,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Data Scientist,nbn™,Sydney NSW,NaN,"As nbn is moving to the scale phase, in parall...",NaN,NaN,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Data Scientist,ResMed,Sydney NSW,NaN,Why ResMed?Imagine what you could accomplish i...,NaN,NaN,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### <a id='2.2.5'>2.2.5 Manually creating skill variables</a>

In [80]:
#Manually creating 'skill' columns

In [81]:
jobs['skill_sql'] = [1 if 'sql' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_python'] = [1 if 'python' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_spark'] = [1 if 'spark' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_hadoop'] = [1 if 'hadoop' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_tableau'] = [1 if 'tableau' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_r'] = [1 if 'r' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_excel'] = [1 if 'excel' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_sas'] = [1 if 'sas' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_spss'] = [1 if 'spss' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_aws'] = [1 if 'aws' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_kafka'] = [1 if 'kafka' in x else 0 for x in jobs['stemmed_summary']]



## <a id='2.3'>2.3 Final cleaning and exporting</a>

In [82]:
#Now that we've made feature columns, we have some remaining columns that won't be useful for our modelling.
#These includes the remaining salary info, the full summary, the stemmed summary and the state and original location column.
#I will keep job title and company for indexing purposes.
dropcols = ['location','salary','summary','state','stemmed_summary']

In [83]:
jobs = jobs.drop(dropcols, axis=1)

In [84]:
jobs.head()

,index,title,company,salary_low,salary_high,salary_mean,title_analyst,title_data,title_scientist,title_business,...,skill_python,skill_spark,skill_hadoop,skill_tableau,skill_r,skill_excel,skill_sas,skill_spss,skill_aws,skill_kafka
0,0,Data Engineer,Telstra,NaN,NaN,NaN,0.000000,0.489781,0.000000,0.0,...,1,1,0,0,1,0,0,0,0,1
1,1,Junior Data Analyst/Scientist,International Institute of Data & Analytics,NaN,NaN,NaN,0.268433,0.339808,0.475133,0.0,...,1,0,0,0,1,1,1,0,0,0
2,2,Data Scientist,Nearmap - AU,NaN,NaN,NaN,0.000000,0.581723,0.813387,0.0,...,1,0,0,0,1,0,0,0,0,0
3,3,Data Scientist,nbn™,NaN,NaN,NaN,0.000000,0.581723,0.813387,0.0,...,0,0,0,0,1,1,0,0,0,0
4,4,Data Scientist,ResMed,NaN,NaN,NaN,0.000000,0.581723,0.813387,0.0,...,0,0,0,0,1,1,0,0,0,0


In [85]:
#Finally, we'll export this to a new .csv file which we can use as a basis for modelling going forward.
filename = 'jobs_clean.csv'
jobs.to_csv(filename)